
# Clase 4: SQL Avanzado

### Introducción
Durante esta clase vamos a aprender los conceptos avanzados de SQL. Vamos a aprender a anidar consultas, a hacer agregación y algunos comandos especiales.

### Requisitos
Para esta actividad, así como en las siguientes actividades de SQL vamos a utilizar

- Python 3
- Jupyter
- La librería ipython-sql. Para instalarla pueden hacerlo con pip3:
```
pip3 install ipython-sql
```
- SQLite3, que viene junto con la instalación de Python

### Outline

En esta clase veremos:

- Consultas anidadas
    - Operadores `IN`, `ALL`, `ANY`, `EXISTS`
    - Anidar tuplas
- Agregación y agrupación
    - `SUM`, `MAX`, `MIN`
    - `GROUP BY`
    - `HAVING`
- `UPDATE`
- `DELETE`

### Agradecimientos

Este _notebook_ fue escrito principalmente por la ayudante **María Ignacia Sánchez**.

### Consultas anidadas

Ya conocemos la manera básica de realizar consultas,  ahora veremos como estas se pueden anidar para realizar consultas más complejas. Primero, cargamos sql, y nos conectamos a la base de datos. Armaremos la base de datos y trataremos de obtener los nombres de todas aquellas bandas cuyos integrantes tengan a alumnos de la UC __y__ toquen en el Lollapalooza:

In [ ]:
%load_ext sql
%sql sqlite:///bdd_sql2.db

In [ ]:
%sql DROP TABLE IF EXISTS bandas 
%sql CREATE TABLE bandas(nombre VARCHAR(10), vocalista VARCHAR(10))

In [ ]:
%sql INSERT INTO bandas VALUES('IIC2413', 'Adrian Soto')
%sql INSERT INTO bandas VALUES('3142CII', 'Naidrán Otos')
%sql INSERT INTO bandas VALUES('Queen', 'Freddie Mercury')
%sql INSERT INTO bandas VALUES('Arctic Monkeys', 'Alex Turner')

In [ ]:
%sql SELECT * FROM bandas

In [ ]:
%sql DROP TABLE IF EXISTS festivales 
%sql CREATE TABLE festivales(nombre VARCHAR(10), banda VARCHAR(10))
%sql INSERT INTO festivales VALUES('Lollapalooza', 'IIC2413')
%sql INSERT INTO festivales VALUES('Lollapalooza', 'IIC2413')
%sql INSERT INTO festivales VALUES('Lollapalooza', '3142CII')
%sql INSERT INTO festivales VALUES('Lollapalooza', 'Arctic Monkeys')
%sql INSERT INTO festivales VALUES('Festival de Viña', 'Queen')
%sql SELECT * FROM festivales


__Ojo:__ Notemos que tenemos un duplicado en esta tabla.

In [ ]:
%sql DROP TABLE IF EXISTS estudiantes_uc  
%sql CREATE TABLE estudiantes_uc(numero_alumno INT, nombre VARCHAR(50))
%sql INSERT INTO estudiantes_uc VALUES(1234567, 'Adrian Soto')
%sql INSERT INTO estudiantes_uc VALUES(7654321, 'Naidrán Otos')
%sql INSERT INTO estudiantes_uc VALUES(1111111, 'Isidora Vizcaya')
%sql INSERT INTO estudiantes_uc VALUES(2222222, 'Ash Ketchum')

%sql SELECT * FROM estudiantes_uc


In [ ]:
%%sql 
SELECT bandas.nombre
FROM bandas, estudiantes_uc
WHERE bandas.vocalista = estudiantes_uc.nombre 
AND bandas.nombre = 
    (SELECT festivales.banda
     FROM festivales
     WHERE festivales.nombre = 'Lollapalooza')

Sin embargo, esta consulta debería retornar más de una banda. ¿Que pasó?
**Hint**: Estamos comparando con igualdad algo que es una colección de valores.

### Operador IN

In [ ]:
%%sql
SELECT bandas.nombre
FROM bandas, estudiantes_uc
WHERE bandas.vocalista = estudiantes_uc.nombre 
AND bandas.nombre IN 
    (SELECT festivales.banda
     FROM festivales
     WHERE festivales.nombre = 'Lollapalooza')

Aquí bandas.nombre está dentro de un listado de valores. ¿Qué pasa si quitamos la anidación?

In [ ]:
%%sql 
SELECT bandas.nombre
FROM bandas, estudiantes_uc, festivales
WHERE bandas.vocalista = estudiantes_uc.nombre 
    AND bandas.nombre = festivales.banda
    AND festivales.nombre = 'Lollapalooza' 

¿Por qué no es equivalente a la consulta anterior? 

__Hint:__ ¡Cuidado con los duplicados! :O

En el caso de subconsulta con duplicados, esta consulta:

In [ ]:
%%sql
SELECT bandas.nombre
FROM bandas, estudiantes_uc
WHERE bandas.vocalista = estudiantes_uc.nombre 
AND bandas.nombre IN 
    (SELECT festivales.banda
     FROM festivales
     WHERE festivales.nombre = 'Lollapalooza')

Es equivalente a  la siguiente consulta, que utiliza `SELECT DISTINCT`:

In [ ]:
%%sql 
SELECT DISTINCT bandas.nombre
FROM bandas, estudiantes_uc, festivales
WHERE bandas.vocalista = estudiantes_uc.nombre 
    AND bandas.nombre = festivales.banda
    AND festivales.nombre = 'Lollapalooza' 

Recordemos que `SELECT DISTINCT` elimina los elementos de la tabla que están duplicados, y sólo entrega valores distintos.

### Operadores ALL y ANY 
__Ojo: Estos no existen en SQLite, pero funcionan con PSQL.__

Para ver qué hacen estos operadores, crearemos otras relaciones:

In [ ]:
%sql DROP TABLE IF EXISTS cervezas
%sql DROP TABLE IF EXISTS distribuidores
%sql CREATE TABLE cervezas(nombre VARCHAR(20), precio INT, id_distribuidor INT)
%sql CREATE TABLE distribuidores(id INT, nombre VARCHAR(20))

In [ ]:
%sql INSERT INTO cervezas VALUES('Quimera', 1190, 0)
%sql INSERT INTO cervezas VALUES('Volcanes del Sur', 1190, 0)
%sql INSERT INTO cervezas VALUES('Mahou', 890, 1)
%sql INSERT INTO cervezas VALUES('Báltica', 90, 2)
%sql INSERT INTO cervezas VALUES('BDD', 70, 1)
%sql INSERT INTO cervezas VALUES('Austral', 1000, 2)

%sql INSERT INTO distribuidores VALUES(0, 'Distribuidora SQLite')
%sql INSERT INTO distribuidores VALUES(1, 'Distribuidora PSQL')
%sql INSERT INTO distribuidores VALUES(2, 'Distribuidora MySQL')

In [ ]:
%sql SELECT * FROM Cervezas

In [ ]:
%sql SELECT * FROM distribuidores

Luego, las cervezas más baratas que la Austral serán:

```SQL
SELECT cervezas.nombre
FROM cervezas
WHERE cervezas.precio < ALL
        (SELECT c2.precio
         FROM cervezas as c2
         WHERE c2.nombre = 'Austral')
```

La cerveza que no sea la más cara será:

```SQL
SELECT cervezas.nombre
FROM cervezas
WHERE cervezas.precio < ANY
        (SELECT c2.precio
         FROM cervezas AS c2)
```

Ahora, ¿podemos expresar estas consultas __solo utilizando `SELECT`, `FROM`, `WHERE` (SFW)__?

__HINT:__ Las consultas SFW son __monótonas__. Una consulta con ALL no es monótona. Una consulta con ANY, lo es.


### Anidando consultas relacionadas

Supongamos que tenemos una relación de la forma peliculas(título, año, director), y queremos sacar los títulos de las películas que se repiten en años diferentes:

```SQL
SELECT p.titulo
FROM peliculas AS p 
WHERE AÑO <> ANY
    (SELECT año
     FROM peliculas
     WHERE titulo = p.titulo)
```

¡__p__ sigue activa a medida que se anidan las consultas!

### Operador EXISTS

(Este operador si funciona en sqlite, por lo que ocuparemos la relacion _cervezas_ que definimos más arriba)

Obtendremos los distribuidores con cervezas más baratas de $100:



In [ ]:
%%sql
SELECT D.nombre
FROM distribuidores AS D
WHERE EXISTS
    (SELECT cervezas.nombre
     FROM cervezas
     WHERE cervezas.id_distribuidor = D.id 
           AND cervezas.precio < 100)

¡Es importante no olvidar el alias para no perder la referencia!
### Anidar tuplas



In [ ]:
%sql DROP TABLE IF EXISTS peliculas
%sql CREATE TABLE peliculas(titulo VARCHAR(20), año INT, director VARCAHAR(20), cine VARCHAR(20))

%sql INSERT INTO peliculas VALUES('Titanic', 1997, 'James Cameron', 'cineplanet')
%sql INSERT INTO peliculas VALUES('El padrino', 1990, 'Francis Ford Coppola', 'cinemark')
%sql INSERT INTO peliculas VALUES('Coco', 2017, 'Lee Unkrich', 'cinemark')
%sql SELECT * FROM peliculas

In [ ]:
%%sql
SELECT titulo
FROM peliculas
WHERE (titulo, año) IN (SELECT titulo, año 
                        FROM peliculas
                        WHERE cine = 'cinemark')

__Ojo__: esto no funciona en todos los sistemas!


Ahora seleccionaremos los productos, junto a sus fabricantes, que son más caros que todos los productos hechos por el mismo fabricante antes del 2019. Consideremos la siguiente relación:

In [ ]:
%sql DROP TABLE IF EXISTS productos
%sql CREATE TABLE productos(nombre VARCHAR(20), precio INT, categoria VARCHAR(20), fabricante VARCHAR(20), año_fabricacion INT)

%sql INSERT INTO productos VALUES('Yogurt Frutilla 225ml', 690, 'Lácteo', 'Nestlé', 2018)
%sql INSERT INTO productos VALUES('Yogurt Frutilla 1L', 990, 'Lácteo', 'Nestlé', 2018)
%sql INSERT INTO productos VALUES('Manjar 500ml', 450, 'Lácteo', 'Nestlé', 2019)
%sql INSERT INTO productos VALUES('Manjar 500ml', 1290, 'Lácteo', 'Nestlé', 2019)
%sql INSERT INTO productos VALUES('Manjar 500ml', 1500, 'Lácteo', 'Colún', 2019)
%sql SELECT * FROM productos


```SQL
SELECT DISTINCT p.nombre, p.fabricante
FROM productos as p 
WHERE p.precio > ALL (SELECT p2.precio 
                      FROM productos as p2
                      WHERE p.fabricante = p2.fabricante 
                            AND p2.año_fabricacion < 2019)
```

### Agregación 
¿Qué hace esta consulta?


In [ ]:
%%sql
SELECT AVG(precio)
FROM productos
WHERE fabricante = 'Nestlé'

También podemos usar `SUM`, `MIN` y `MAX` y usar __operaciones aritméticas__:

In [ ]:
%sql DROP TABLE IF EXISTS compra
%sql CREATE TABLE compra(producto VARCHAR(20), fecha VARCHAR(20), precio INT, cantidad INT)
%sql INSERT INTO compra VALUES('Leche', '01-02-2019', 890, 4)
%sql INSERT INTO compra VALUES('Chocapic', '01-02-2019', 1290, 2)
%sql INSERT INTO compra VALUES('Queso' , '01-02-2019', 990, 1)
%sql INSERT INTO compra VALUES('Pan' , '01-02-2019', 1190, 1)
%sql SELECT * FROM compra

### Suma

In [ ]:
%%sql
SELECT SUM(precio*cantidad)
FROM compra
WHERE producto = 'Leche'

### Contar tuplas:
El comano __COUNT__ cuenta las tuplas. __OJO:__ Se cuentan los duplicados!

In [ ]:
%%sql
SELECT COUNT(*) as cuenta_total
FROM productos
WHERE año_fabricacion > 2018

La consulta anterior se comporta igual que esta: 

In [ ]:
%%sql
SELECT COUNT(fabricante) as cuenta_total
FROM productos
WHERE año_fabricacion > 2018

### GROUP BY
Contaremos los prouctos de cada fabricante, utilizando el operador __GROUP BY__

In [ ]:
%%sql 
SELECT fabricante, COUNT(fabricante)
FROM productos
WHERE año_fabricacion > 2018
GROUP BY fabricante

Esta consulta:
    - Computa los resultados según el FROM y el WHERE 
    - Agrupa los resultados según los atributos del GROUP BY 
    - Para cada grupo se aplica inependientemente la agregación 
Otro ejemplo:

In [ ]:
%sql DROP TABLE IF EXISTS compra
%sql CREATE TABLE compra(producto VARCHAR(20), fecha TEXT, precio INT, cantidad INT)
%sql INSERT INTO compra VALUES('tomates', '2018-02-07', 100, 6)
%sql INSERT INTO compra VALUES('tomates', '2018-07-06', 150, 4)
%sql INSERT INTO compra VALUES('zapallos', '2018-02-08', 800, 1)
%sql INSERT INTO compra VALUES('zapallos', '2018-07-09', 1000, 2)
%sql INSERT INTO compra VALUES('zapallos', '2018-01-01', 600, 3)
%sql SELECT * FROM compra

In [ ]:
%%sql
SELECT producto, SUM(precio*cantidad) AS ventaTotal
FROM compra 
WHERE fecha > '2018-01-10'
GROUP BY producto

¿Cómo funciona esto?. En primer lugar, se ejecuta el from y el where:

<table>
  <tr>
    <th>Producto</th>
    <th>Fecha</th> 
    <th>Precio</th>
    <th>Cantidad</th>
  </tr>
  <tr>
    <td>Tomates</td>
    <td>07/02</td> 
    <td>100</td>
    <td>6</td>
  </tr>
  <tr>
    <td>Tomates</td>
    <td>06/07</td> 
    <td>150</td>
    <td>4</td>
  </tr>
  <tr>
    <td>Zapallo</td>
    <td>08/02</td> 
    <td>800</td>
    <td>1</td>
  </tr>
  <tr>
    <td>Zapallo</td>
    <td>09/07</td> 
    <td>1000</td>
    <td>2</td>
  </tr>
</table>

Luego, se agrupa según el group by:

<table>
  <tr>
    <th>Producto</th>
    <th>Fecha</th> 
    <th>Precio</th>
    <th>Cantidad</th>
  </tr>
  <tr>
    <td rowspan="2">Tomates</td>
    <td>07/02</td> 
    <td>100</td>
    <td>6</td>
  </tr>
  <tr>
    <td>06/07</td> 
    <td>150</td>
    <td>4</td>
  </tr>
  <tr>
    <td rowspan="2">Zapallo</td>
    <td>08/02</td> 
    <td>800</td>
    <td>1</td>
  </tr>
  <tr>
    <td>09/07</td> 
    <td>1000</td>
    <td>2</td>
  </tr>
</table>

Finalmente, se agrega por grupo y se ejecuta la proyección:

<table>
  <tr>
    <th>Producto</th>
    <th>VentaTotal</th>
  </tr>
  <tr>
    <td>Tomates</td>
    <td>1200</td>
  </tr>
  <tr>
    <td>Zapallo</td>
    <td>2800</td>
  </tr>
</table>

### HAVING

Consideremos ahora la misma consulta, pero sólo considerando aquellos productos que se vendieron más de 100 veces (Para mostrar el efecto de este operadore, agregaremos un par de filas a la tabla)

In [ ]:
%sql INSERT INTO compra VALUES('zanahorias', '2018-07-06', 150, 125)
%sql INSERT INTO compra VALUES('zanahorias', '2018-02-08', 800, 50)

In [ ]:
%sql SELECT * FROM compra

In [ ]:
%%sql
SELECT producto, SUM(precio*cantidad) AS ventaTotal, sum(cantidad)
FROM compra 
WHERE fecha > '2018-01-10'
GROUP BY producto
HAVING sum(cantidad) > 100

¿Por qué usamos `HAVING` y no lo incluimos en el `WHERE`?

### Consultas con agregación

```SQL
SELECT <S>
FROM R_1, ..., R_N
WHERE <Condición 1>
GROUP BY a_1, ..., a_k
HAVING <Condición 2>
```

- $S$ puede contener atributos $a_1, ..., a_k$ y/o agregados, pero ningún otro atributo (¿Por qué?)
- _Condición 1_ es una condición que usa atributos $R_1, ..., R_n$
- _Condición 2_ es una condición de agregación de los atributos $R_1, ..., R_n$

#### ¿Cómo se evalúan este tipo de consultas?

1. Se computa el `FROM` - `WHERE` de $R_1, ..., R_n$
2. Se agrupa la tabla por los atributos de $a_1, ..., a_k$
3. Se computan los agregados de la _Condición 2_ y se mantienen los grupos que la satisfacen
4. Se computan los agregados de $S$ y se entrega el resultado.

Armemos un ejemplo:

In [ ]:
%sql DROP TABLE IF EXISTS Autor
%sql DROP TABLE IF EXISTS Documento
%sql DROP TABLE IF EXISTS Escribe
%sql DROP TABLE IF EXISTS Menciona

%sql CREATE TABLE Autor(login INT, nombre VARCHAR(20))
%sql CREATE TABLE Documento(id_doc VARCHAR(30), titulo VARCHAR(20))
%sql CREATE TABLE Escribe(login INT, id_doc VARCHAR(30))
%sql CREATE TABLE Menciona(id_doc VARCHAR(30), palabra VARCHAR(20))

%sql INSERT INTO Autor VALUES(0, 'Marcela Paz')
%sql INSERT INTO Autor VALUES(1, 'Pablo Neruda')
%sql INSERT INTO Autor VALUES(2, 'Gabriela Mistral')
%sql INSERT INTO Documento VALUES(0, 'Papelucho')
%sql INSERT INTO Documento VALUES(1, 'Papelucho en la clínica')
%sql INSERT INTO Documento VALUES(2, 'Papelucho detective')
%sql INSERT INTO Documento VALUES(3, 'Papelucho perdido')
%sql INSERT INTO Escribe VALUES(0,0)
%sql INSERT INTO Escribe VALUES(0,1)
%sql INSERT INTO Escribe VALUES(0,2)
%sql INSERT INTO Escribe VALUES(0,3)
%sql INSERT INTO Documento VALUES(4, 'Cien Sonetos de Amor')
%sql INSERT INTO Documento VALUES(5, 'Confieso que he vivido')
%sql INSERT INTO Escribe VALUES(1,4)
%sql INSERT INTO Escribe VALUES(1,5)
%sql INSERT INTO Documento VALUES(6, 'Locas Mujeres')
%sql INSERT INTO Documento VALUES(7, 'Poemas')
%sql INSERT INTO Documento VALUES(8, 'Desolación')
%sql INSERT INTO Escribe VALUES(2,6)
%sql INSERT INTO Escribe VALUES(2,7)
%sql INSERT INTO Escribe VALUES(2,8)


Encontremos a todos los autores que escribieron al menos 3 documentos:

In [ ]:
%%sql
SELECT DISTINCT *
FROM (SELECT Autor.login, COUNT(*) as countAutor
      FROM Escribe, Autor
      WHERE Escribe.login = Autor.login
      GROUP BY Autor.login) AS Foo
WHERE countAutor >= 3

¡La respuesta puede mejorar! Estamos haciendo una anidación innecesaria.

__Obs:__ El uso del alias (en este caso: Foo) en el FROM es necesario para el uso de subconsultas

Veamos una versión más elegante:

In [ ]:
%%sql
SELECT Autor.nombre
FROM Autor, Escribe
WHERE Autor.login = Escribe.login
GROUP BY Autor.nombre
HAVING COUNT(Escribe.id_doc) >= 3

__NOTA:__ No necesitamos __DISTINCT__ gracias al GROUP BY :D 

Otro ejemplo: Encontrar a todos los autores con un vocabulario de más de 10000 palabras. 

```SQL
SELECT Autor.nombre
FROM Autor, Escribe, Menciona
WHERE Autor.login = Escribe.login AND Escribe.id_doc = Menciona.id_doc
GROUP BY Autor.nombre
HAVING COUNT(DISTINCT Menciona.palabra ) >= 10000
```

### INSERT
Para insertar valores usamos:

```SQL
INSERT INTO R
VALUES <Valores>
```

Tambinén podemos insertar valores de otra consulta. Por ejemplo, imaginemos que tenemos una lista con la programación del día de hoy, y una lista de series:


In [ ]:
%sql DROP TABLE IF EXISTS Programacion
%sql DROP TABLE IF EXISTS Series

%sql CREATE TABLE Programacion(titulo VARCHAR(20), hora TXT, duracion_min INT)
%sql CREATE TABLE Series(titulo)

%sql INSERT INTO Programacion VALUES('Hora de Aventura', '22:00', 15)
%sql INSERT INTO Programacion VALUES('Shingeki No Kyojin', '22:20', 24)
%sql INSERT INTO Programacion VALUES('Juego de Tronos', '22:50', 40)
%sql INSERT INTO Series VALUES('Juego de Tronos')
%sql INSERT INTO Series VALUES('Hora de Aventura')
%sql SELECT * FROM Series


In [ ]:
%sql SELECT * FROM Programacion

In [ ]:
%%sql 
SELECT DISTINCT titulo
FROM Programacion
WHERE Programacion.titulo NOT IN (SELECT titulo FROM Series)
        

La consulta anterior nos arroja el resultado que nos interesaría insertar en la lista de series, luego:

In [ ]:
%%sql 
INSERT INTO Series SELECT DISTINCT titulo
                          FROM Programacion
                          WHERE Programacion.titulo NOT IN (SELECT titulo FROM Series)

In [ ]:
%sql SELECT * FROM Series

Estamos insertando sólo el título ¿Qué pasa con los demás valores?

### UPDATE
Para actualizar valores de una tabla usamos:

```SQL
UPDATE R
SET <Nuevos valores>
WHERE <Condición sobre R>
```

Donde: 

```
<Nuevos valores> := (atributo_1 = nuevoValor_1, ..., atributo_n = nuevoValor_n)
```

Por ejemplo, consideremos la siguiente relacion:


In [ ]:
%sql DROP TABLE IF EXISTS Estudiantes 
%sql CREATE TABLE Estudiantes(nombre VARCHAR(20), apellido VARCHAR(20), rut VARCHAR(20))
%sql INSERT INTO Estudiantes VALUES('John','Lennon', '12333222K')
%sql INSERT INTO Estudiantes VALUES('Paul','McCartney', '422235554')
%sql INSERT INTO Estudiantes VALUES('Richard','Starkey', '22222222K')
%sql SELECT * FROM Estudiantes


In [ ]:
%%sql
UPDATE Estudiantes 
SET nombre = 'apellido' || 'nombre'
WHERE rut LIKE '%K%'


In [ ]:
%sql SELECT * FROM Estudiantes

Otro operador que podemos usar es:

```SQL
ALTER TABLE Estudiantes DROP apellido
```

### DELETE
Para borrar tuplas que satisfagan una condición, usamos: 

```DELETE FROM R
 WHERE <Condición sobre R>```

Por ejemplo:

In [ ]:
%sql DELETE FROM Estudiantes WHERE Rut LIKE '%K%'
%sql SELECT * FROM Estudiantes 

Para borrar todo: 

In [ ]:
%sql DELETE FROM Estudiantes 
%sql SELECT * FROM Estudiantes